In [1]:
import cv2 
import numpy as np
import time
import pyrealsense2 as rs

In [11]:
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
if depth_sensor.supports(rs.option.depth_units):
    depth_sensor.set_option(rs.option.depth_units,0.001)
depth_scale = depth_sensor.get_depth_scale()
align = rs.align(rs.stream.color)

In [10]:
pipeline.stop()

In [3]:
frames = pipeline.wait_for_frames()
aligned_frames = align.process(frames)   
depth_frame = aligned_frames.get_depth_frame() 
color_frame = aligned_frames.get_color_frame()
frame = np.asanyarray(color_frame.get_data())

In [5]:
depth_frame.get_distance(360, 640)

0.3660000264644623

In [12]:
while True:
    frames = pipeline.wait_for_frames()
    if not frames:
        break

    aligned_frames = align.process(frames)   
    depth_frame = aligned_frames.get_depth_frame() 
    color_frame = aligned_frames.get_color_frame()
    frame = np.asanyarray(color_frame.get_data())
    cv2.imshow("Realsense RGB", frame)

    key = cv2.waitKey(1)
    if key == ord('s') or key == ord('S'):
        now = time.localtime(time.time())
        y = str(now.tm_year)
        m = str(now.tm_mon) if now.tm_mon > 9 else '0' + str(now.tm_mon)
        h = str(now.tm_hour) if now.tm_hour > 9 else '0' + str(now.tm_hour)
        u = str(now.tm_min) if now.tm_min > 9 else '0' + str(now.tm_min)
        s = str(now.tm_sec) if now.tm_sec > 9 else '0' + str(now.tm_sec)
        cv2.imwrite("{}{}{}{}{}_Kebuke.jpg".format(y, m, h, u, s), frame)
        
    if key == 27:  # ESC key: quit program
        cv2.destroyAllWindows()
        break

In [6]:
sift_detector = cv2.SIFT_create()
img = cv2.imread("3kbk_Color.png")
aging = cv2.imread("aging1.png")
kpts_img, des_img = sift_detector.detectAndCompute(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), None)
kpts_aging, des_aging = sift_detector.detectAndCompute(cv2.cvtColor(aging, cv2.COLOR_BGR2GRAY), None)

In [16]:
kpts_bimg, des_bimg = sift_detector.detectAndCompute(cv2.GaussianBlur(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), ksize=(5, 5), sigmaX=0.7), None)
kpts_baging, des_baging = sift_detector.detectAndCompute(cv2.GaussianBlur(cv2.cvtColor(aging, cv2.COLOR_BGR2GRAY), ksize=(5, 5), sigmaX=0.7), None)

In [18]:
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des_bimg,des_baging,k=2)
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.6*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
img3 = cv2.drawMatchesKnn(img,kpts_bimg,aging,kpts_baging,matches,None,**draw_params)
cv2.imshow("ouo", img3)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
cv2.imshow("ouo", img)
cv2.waitKey()
cv2.destroyAllWindows()